<a href="https://colab.research.google.com/github/hansglick/book_errata/blob/main/p003_tfhub_usage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
print(tf.__version__)

2.8.0


In [28]:
# J'ai pas super bien compris le découpage
# Apparemment les trois sorties sont des prefetch dataset
# Apparemment on peut sortir un batch de taille souhaitable en appelant .batch(taille)
# Les donnée sont déjà ordonnées, donc prendre .batch(1) est contenu dans .batch(2)
# Je vois aps trop l'intérêt
# C'est une classe un peu particulière de data structure: plus d'informations ici : https://www.tensorflow.org/jvm/api_docs/java/org/tensorflow/op/data/PrefetchDataset
a = "train[:60%]"
b  ="train[:60%]"
c = "test"
train,test,validation = tfds.load(name="imdb_reviews",
                                  split=(a,b,c),
                                  as_supervised=True)

In [29]:
# Les exemples et labels sont des tensor classiques
exemples,labels = next(iter(train.batch(2)))

In [35]:
# Les modèles de TF Hub sont wrappables dans un hub.KerasLayer()
# L'output est de type tf.keras layer classique
# On a à priori, pas besoin, de renseigner input_shape, le modèle prend des phrases de n'importe quelle taille
embedding_model_path = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding_model_path,
                           input_shape=[],
                           dtype=tf.string,
                           trainable=True)

# Application du layer à des données
# Chaque phrase est encodé dans un embedding de taille 50
hub_layer(exemples)

In [38]:
# Architecture du modèle
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16,activation = "relu"))
model.add(tf.keras.layers.Dense(1))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 50)                48190600  
                                                                 
 dense (Dense)               (None, 16)                816       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


In [39]:
# Compilation du modèle
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [40]:
# Modèle entraînement
history = model.fit(train.shuffle(10000).batch(512),
                    epochs=10,
                    validation_data=validation.batch(512),
                    verbose=1)

Epoch 1/10
30/30 [==============================] - 12s 370ms/step - loss: 0.6848 - accuracy: 0.5007 - val_loss: 0.6643 - val_accuracy: 0.5001
Epoch 2/10
30/30 [==============================] - 11s 359ms/step - loss: 0.6279 - accuracy: 0.5341 - val_loss: 0.5934 - val_accuracy: 0.6088
Epoch 3/10
30/30 [==============================] - 11s 365ms/step - loss: 0.5145 - accuracy: 0.7159 - val_loss: 0.4832 - val_accuracy: 0.7432
Epoch 4/10
30/30 [==============================] - 12s 387ms/step - loss: 0.3791 - accuracy: 0.8391 - val_loss: 0.3933 - val_accuracy: 0.8112
Epoch 5/10
30/30 [==============================] - 11s 363ms/step - loss: 0.2691 - accuracy: 0.8989 - val_loss: 0.3436 - val_accuracy: 0.8389
Epoch 6/10
30/30 [==============================] - 11s 366ms/step - loss: 0.1896 - accuracy: 0.9340 - val_loss: 0.3245 - val_accuracy: 0.8512
Epoch 7/10
30/30 [==============================] - 12s 380ms/step - loss: 0.1356 - accuracy: 0.9589 - val_loss: 0.3218 - val_accuracy: 0.8551

In [47]:
# Evaluation du modele
# results est une liste qui contient les métriques d'évaluation
# le label des metriques est accessible dans model.metrics_name
results = model.evaluate(test.batch(2048), verbose=2)
print(model.metrics_names)
print(results)

8/8 - 1s - loss: 0.0411 - accuracy: 0.9929 - 1s/epoch - 184ms/step
